In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os import path as op
import itertools
import re
import xarray as xr
from sklearn.model_selection import train_test_split
import numpy.random as nr

%matplotlib inline

/scratch/adomakor412/conda/envs/MyEnv/lib/python3.6/site-packages/dask/config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
nr.seed = 42

In [3]:
inputPath = '/scratch/adomakor412/all_npy3'
PATH = os.path.expanduser(inputPath)
ncPath = os.path.expanduser('/scratch/adomakor412/april_data_2019')

In [4]:
columns = [
    "time",
    "longitude",
    "latitude",
    #"band",#Necessary?
    "G17_Temp",
    "G17_mean",
    "G17_std",#mean and std outside inner for loop for comp. eff.
    "target_G16_Temp"
]

df = pd.DataFrame({},columns = columns)

In [5]:
def longitude(lonMin, lonMax, col, colSize):
    lon = (col/colSize)* (lonMax - lonMin)
    return lon

def latitude(latMin, latMax, row, rowSize):
    lat = (row/rowSize)* (latMax - latMin)
    return lat

def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    invRad = np.array(rad)**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2) 
    return T

In [6]:
import logging
logger = logging.getLogger()

In [7]:
extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]

dataFrames = []
randPool = 401401
randSize = 1000#randPool*.1
inds=nr.randint(randPool, size=(randSize))
inds.sort()

In [ ]:
MAE = []
MSE = []
R2 = []
R2_train_test = []

dfFixedTest = []

trainSamp = [100, 1000, 10000, 100000]
for samp in trainSamp:
    with open('logML', 'a') as log:
        dataFrames = []
        for (bb,dd) in list(itertools.product([8],[5])):#Let's start with one day
            DD = str(98+dd).zfill(3)
            lookup = f'M6C08_G17_s2019{DD}0000'
            ncFiles = [f for f in os.listdir(ncPath) if re.search(lookup,f)]
            #ncFiles = mySort(ncFiles)
            npFiles = [f for f in os.listdir( PATH ) if re.search(lookup,f)]
            #ncFiles = mySort(ncFiles)

            for ncf, npf in zip(ncFiles,npFiles):#can refactor to just conversion of selected files
                try:
                    imageBox = np.load(op.join( PATH,npf))
                    myFile = xr.open_dataset(op.join(ncPath,ncf))
                    planck_fk1 = float(myFile['planck_fk1'].data)
                    planck_fk2 = float(myFile['planck_fk2'].data) 
                    planck_bc1 = float(myFile['planck_bc1'].data)                       
                    planck_bc2 = float(myFile['planck_bc2'].data)

                    time = ncf[31:38]
                    G17_mean = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
                    G17_std = Rad2BT(imageBox.std(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)

                    hh = ncf[34:36]
                    mm = ncf[36:38]

                    print(ncf)
                    print(str(ncf), file=log)
                    logger.info(str(ncf))

                    G16_npy = np.load( op.join(PATH, npf.replace('G16','G17',1)) )
                    G16_ncf = xr.open_dataset(op.join( ncPath, ncf.replace('G16','G17',1) ))
                    G16_fk1 = float(G16_ncf['planck_fk1'].data)
                    G16_fk2 = float(G16_ncf['planck_fk2'].data) 
                    G16_bc1 = float(G16_ncf['planck_bc1'].data)                       
                    G16_bc2 = float(G16_ncf['planck_bc2'].data)

                    target_G16_Temp = Rad2BT(G16_npy, G16_fk1, G16_fk2, G16_bc1, G16_bc2)
                    x,y = imageBox.shape[0],imageBox.shape[1]
                    randInds = nr.randint(10000, size=(x*y))

                    #select "10,000" random points out of 401,401
                    sample = np.array([combo for combo in itertools.product(range(x),range(y))])
                    for i,j in sample:
                        lon = longitude( extent_pc[0], extent_pc[1], i, x )
                        lat = latitude( extent_pc[2], extent_pc[3], j, y)

                        G17_Temp = Rad2BT(imageBox[i,j], planck_fk1, planck_fk2, planck_bc1, planck_bc2)#unfiltered
                        target_G16_Temp = Rad2BT( G16_npy[i,j], G16_fk1, G16_fk2, G16_bc1, G16_bc2 )

                        row = [time, lon, lat, G17_Temp, G17_mean, G17_std, target_G16_Temp]

                        dataFrames.append(pd.DataFrame([row],columns=columns))
                        #concatenate a list of single dataframes after done with loop for speed " pandas.concat"
                        #z score versus the standard deviation for single image, can normalize across all image

                except ValueError as e:
                    logger.exception(e)
                    print(e)
                    print(e, file=log)
                    continue
                 
        
        if len(dfFixedTest)==0:
            dfFixedTest = df.sample(1000)
            X_test = dfFixedTest.drop(["target_G16_Temp"],axis=1).astype('float')
            y_test = np.ravel(dfFixedTest.drop(columns[:-1],axis=1).astype('float'))

        train = regr.score(X_train,y_train)
        test = regr.score(X_test, y_test)

        R2.append(train)
        R2_train_test.append(train/test)

        prediction = regr.predict(X_train)
        errors = abs(prediction - y_train)

        MAE.append( round(np.mean(errors), 2))

        MSE.append(mean_squared_error(prediction, y_test))
        
        print('MAE', MAE)
        print('MSE', MSE)
        print('R2',R2)
        print('R2 train to test ratio',R2_train_test)
# df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", names=columns)
# for column in df.columns:
#     if df[column].dtype == "object":
#         df[column] = df[column].str.strip()
# df = df.dropna()

OR_ABI-L1b-RadF-M6C08_G17_s20191030000339_e20191030009405_c20191030009441.nc


In [ ]:
print('MAE', MAE)
print('MSE', MSE)
print('R2',R2)
print('R2 train to test ratio',R2_train_test)

In [ ]:
inputPath = '/scratch/adomakor412/july_26-27_2019'
PATH = os.path.expanduser(inputPath)
ncPath = os.path.expanduser('/scratch/adomakor412/july_26-27_2019')

MAE_ML = []
MSE_ML = []
R2_ML = []
R2_ML_train_test = []

MAE_pCal = []
MSE_pCal = []
R2_pCal = []
R2_pCal_train_test = []

dfFixedTest_ML = []#df.iloc[[inds]]
dfFixedTest_pCal = []#

trainSamp = [100, 1000, 10000, 100000]
for samp in trainSamp:
    dataFrames = []
    with open('logML', 'a') as log:

        for (bb,dd,SS) in list(itertools.product([8],[207,208],[17])):
            DD = str(dd).zfill(3)
            lookup = f'M6C08_G{SS}_s2019{DD}'
            ncFiles = [f for f in os.listdir(ncPath) if re.search(lookup,f)]
            #ncFiles = mySort(ncFiles)
            npFiles = [f for f in os.listdir( PATH ) if re.search(lookup,f)]
            #ncFiles = mySort(ncFiles)
            
            randInds = nr.randint(samp, size=(x*y))
            randInds.sort()

            for ncf, npf in zip(ncFiles,npFiles):#for ncf, npf in zip(ncFiles,npFiles)[randInds] day sample <=576
                try:
                    imageBox = np.load(op.join( PATH,npf))
                    myFile = xr.open_dataset(op.join(ncPath,ncf))
                    planck_fk1 = float(myFile['planck_fk1'].data)
                    planck_fk2 = float(myFile['planck_fk2'].data) 
                    planck_bc1 = float(myFile['planck_bc1'].data)                       
                    planck_bc2 = float(myFile['planck_bc2'].data)

                    time = ncf[31:38]
                    G17_mean = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
                    G17_std = Rad2BT(imageBox.std(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)

                    hh = ncf[34:36]
                    mm = ncf[36:38]

                    print(ncf)
                    print(str(ncf), file=log)
                    logger.info(str(ncf))

                    G16_npy = np.load( op.join(PATH, npf.replace('G16','G17',1)) )
                    G16_ncf = xr.open_dataset(op.join( ncPath, ncf.replace('G16','G17',1) ))
                    G16_fk1 = float(G16_ncf['planck_fk1'].data)
                    G16_fk2 = float(G16_ncf['planck_fk2'].data) 
                    G16_bc1 = float(G16_ncf['planck_bc1'].data)                       
                    G16_bc2 = float(G16_ncf['planck_bc2'].data)

                    target_G16_Temp = Rad2BT(G16_npy, G16_fk1, G16_fk2, G16_bc1, G16_bc2)
                    x,y = imageBox.shape[0],imageBox.shape[1]

                    #select "10,000" random points out of 401,401
                    sample = np.array([combo for combo in itertools.product(range(x),range(y))])
                    for i,j in sample:
                        lon = longitude( extent_pc[0], extent_pc[1], i, x )
                        lat = latitude( extent_pc[2], extent_pc[3], j, y)

                        G17_Temp = Rad2BT(imageBox[i,j], planck_fk1, planck_fk2, planck_bc1, planck_bc2)#unfiltered
                        target_G16_Temp = Rad2BT( G16_npy[i,j], G16_fk1, G16_fk2, G16_bc1, G16_bc2 )

                        row = [time, lon, lat, G17_Temp, G17_mean, G17_std, target_G16_Temp]

                        dataFrames.append(pd.DataFrame([row],columns=columns))
                        #concatenate a list of single dataframes after done with loop for speed " pandas.concat"
                        #z score versus the standard deviation for single image, can normalize across all image

                except ValueError as e:
                    logger.exception(e)
                    print(e)
                    print(e, file=log)
                    continue
        df=pd.concat(dataFrames)
        df = df.dropna()
        
        from sklearn.neural_network import MLPRegressor
        #from sklearn.datasets import make_regression
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import mean_squared_error
        
        dfSample = df.sample(n=samp)
        X = dfSample.drop(["target_G16_Temp"],axis=1).astype('float')#try as float
        y = dfSample.drop(columns[:-1],axis=1).astype('float')
            
        from sklearn.metrics import mean_squared_error
        X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y),random_state=1,test_size=0.0)
        regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
        
        
        
        If dd == 207:#non pCal
            if len(dfFixedTest_ML)==0:
                dfFixedTest_ML = df.sample(1000)
                X_test = dfFixedTest_ML.drop(["target_G16_Temp"],axis=1).astype('float')
                y_test = np.ravel(dfFixedTest_ML.drop(columns[:-1],axis=1).astype('float'))
            
            train = regr.score(X_train,y_train)
            test = regr.score(X_test, y_test)
            
            R2_ML.append(train)
            R2_ML_train_test.append(train/test)
            
            prediction = regr.predict(X_train)
            errors = abs(prediction - y_train)
            
            MAE = round(np.mean(errors), 2)
            MAE_ML.append(MAE)
            
            MSE = mean_squared_error(prediction, y_test)
            MSE_ML.append(MSE)
            
        If dd == 208:#pCal        
            if len(dfFixedTest_pCal)==0:
                dfFixedTest_pCal = df.sample(1000)
                X_test = dfFixedTest_pCal.drop(["target_G16_Temp"],axis=1).astype('float')
                y_test = np.ravel(dfFixedTest_pCal.drop(columns[:-1],axis=1).astype('float'))
            
            train = regr.score(X_train,y_train)
            test = regr.score(X_test, y_test)
            
            R2_pCal.append(train)
            R2_pCal_train_test.append(train/test)
            
            prediction = regr.predict(X_train)
            errors = abs(prediction - y_train)
            
            MAE = round(np.mean(errors), 2)
            MAE_pCal.append(MAE)
            
            MSE = mean_squared_error(prediction, y_test)
            MSE_pCal.append(MSE)
            
            
            
        
        

In [ ]:
df=pd.concat(dataFrames)
df = df.dropna()

In [ ]:
from sklearn.neural_network import MLPRegressor
#from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
dfSample = df.sample(n=10000)
X = dfSample.drop(["target_G16_Temp"],axis=1).astype('float')#try as float
y = dfSample.drop(columns[:-1],axis=1).astype('float')

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y),random_state=1,test_size=0.0)
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)

In [ ]:
X_test = dfFixedTest.drop(["target_G16_Temp"],axis=1).astype('float')
y_test = np.ravel(dfFixedTest.drop(columns[:-1],axis=1).astype('float'))

In [ ]:
regr.score(X_test, y_test)

In [ ]:
prediction = regr.predict(X_test)

In [ ]:
errors = abs(prediction - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'Kelvin')
print('Mean Square Error:', mean_squared_error(prediction, y_test))

In [ ]:
regr.score(X_train,y_train)

### Previous MSE was 23.96 at 1000 points sampled. Going from 1000 to 10000 points acheived better MSE (23.96 -> 0.36), MAE (3.02 Kelvins -> 0.52 Kelvins) and R^2 of both the test (-0.42 -> 0.61) and train (-0.32 ->0.6) data.

### Since the R^2 of the train and test data are similar, we know we are not overfitting.

In [ ]:
list(itertools.product([8],[207,208],[16,17]))